In [ ]:
# ============================================
#   DEMO DIDÁCTICA DE UN MINI SSH EN PYTHON
# ============================================

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.fernet import Fernet


# ============================================
#             SERVIDOR DIDÁCTICO
# ============================================
class ServidorSSH:
    def __init__(self):
        print("🛠️ Servidor: Generando claves RSA...")

        # Crear un par de claves RSA  (Rivest-Shamir-Adleman).
        # Esta es CRIPTOGRAFÍA ASIMÉTRICA:
        # - private_key: clave privada (NO se comparte)
        # - public_key: clave pública (se comparte)
        # SSH real también usa claves públicas, pero RSA no siempre.
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,  # valor estándar en RSA
            key_size=2048           # tamaño seguro hoy en día
        )

        # Extraer clave pública desde la clave privada
        self.public_key = self.private_key.public_key()

        # Variables para almacenar la clave simétrica de sesión
        self.session_key = None
        self.cipher = None

    def obtener_clave_publica(self):
        """Devuelve la clave pública.
        El cliente la recibirá y la usará para cifrar.
        """

        return self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,               # Formato de texto
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )

    def recibir_clave_simetrica(self, clave_cifrada):
        """Recibe la clave simétrica cifrada por el cliente."""
        print("🛠️ Servidor: Descifrando clave simétrica enviada por el cliente...")

        # Aquí el servidor DESCIFRA usando su CLAVE PRIVADA RSA.
        # Esto demuestra:
        #  - Solo el servidor podía descifrar la clave simétrica
        self.session_key = self.private_key.decrypt(
            clave_cifrada,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

        # Crear un cifrador simétrico Fernet
        # Aquí empieza el canal seguro.
        self.cipher = Fernet(self.session_key)

        print("🔐 Servidor: Clave de sesión establecida con éxito.\n")

    def procesar_comando(self, comando_cifrado):
        """El servidor descifra el comando, procesa una respuesta y la cifra."""

        # Descifrado simétrico con la clave de sesión.
        # Esta es CRIPTOGRAFÍA SIMÉTRICA.
        comando = self.cipher.decrypt(comando_cifrado).decode()
        print(f"📥 Servidor recibió comando:   {comando}")

        # Respuesta simulada (en SSH real aquí se ejecuta en la terminal)
        respuesta = f"Ejecutado correctamente → {comando}"

        # Devuelve la respuesta cifrada simétricamente
        return self.cipher.encrypt(respuesta.encode())


# ============================================
#                 CLIENTE
# ============================================
class ClienteSSH:
    def __init__(self, clave_publica_servidor):
        print("📡 Cliente: Cargando clave pública del servidor...")

        # Carga de la clave pública del servidor.
        # Esta clave se usará SOLO para cifrar.
        # El cliente NO puede descifrar lo que cifra con ella.
        self.server_public_key = serialization.load_pem_public_key(
            clave_publica_servidor
        )

        # Crear clave simétrica para la sesión segura.
        # Esta clave será compartida SOLO UNA VEZ.
        print("📡 Cliente: Generando clave de sesión (simétrica AES/Fernet)...")
        self.session_key = Fernet.generate_key()
        self.cipher = Fernet(self.session_key)

    def cifrar_clave_simetrica(self):
        """Cifra la clave simétrica usando RSA (clave pública del servidor)."""

        print("📡 Cliente: Cifrando clave de sesión con RSA...")

        # Aquí el cliente usa la CLAVE PÚBLICA del servidor para cifrar.
        # Solo el servidor con su CLAVE PRIVADA puede descifrar.
        # Esto es criptografía ASIMÉTRICA.
        return self.server_public_key.encrypt(
            self.session_key,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

    def enviar_comando(self, texto):
        """Cifra un comando con la clave simétrica y lo envía."""

        print(f"\n📤 Cliente enviando comando cifrado:   {texto}")

        # Esto usa CIFRADO SIMÉTRICO (rápido), típico en SSH.
        return self.cipher.encrypt(texto.encode())

    def recibir_respuesta(self, respuesta_cifrada):
        """Descifra la respuesta usando la clave simétrica."""

        respuesta = self.cipher.decrypt(respuesta_cifrada).decode()
        print(f"📥 Cliente recibió respuesta:   {respuesta}")

        return respuesta



# ===================================================
#           SIMULACIÓN TIPO SSH COMPLETA
# ===================================================

print("==============================================")
print("           SIMULACIÓN DE MINI SSH")
print("==============================================\n")

# 1) Crear servidor y generar claves RSA
servidor = ServidorSSH()

# 2) Cliente recibe la clave pública del servidor (como hace SSH real con las host keys)
clave_publica = servidor.obtener_clave_publica()

# 3) Crear cliente con la clave pública del servidor
cliente = ClienteSSH(clave_publica)

# 4) Cliente cifra su clave simétrica y la "manda" al servidor
clave_simetrica_cifrada = cliente.cifrar_clave_simetrica()

# 5) Servidor descifra la clave simétrica y establece la sesión
servidor.recibir_clave_simetrica(clave_simetrica_cifrada)

# 6) Cliente envía un comando cifrado
comando = cliente.enviar_comando("ls -al")

# 7) Servidor procesa y responde
respuesta_cifrada = servidor.procesar_comando(comando)

# 8) Cliente descifra la respuesta
cliente.recibir_respuesta(respuesta_cifrada)

print("\n🔚 FIN DE LA DEMO")

           SIMULACIÓN DE MINI SSH

🛠️ Servidor: Generando claves RSA...
📡 Cliente: Cargando clave pública del servidor...
📡 Cliente: Generando clave de sesión (simétrica AES/Fernet)...
📡 Cliente: Cifrando clave de sesión con RSA...
🛠️ Servidor: Descifrando clave simétrica enviada por el cliente...
🔐 Servidor: Clave de sesión establecida con éxito.


📤 Cliente enviando comando cifrado:   ls -al
📥 Servidor recibió comando:   ls -al
📥 Cliente recibió respuesta:   Ejecutado correctamente → ls -al

🔚 FIN DE LA DEMO
